In [216]:
import torch
from tqdm import tqdm
import os
import cv2
import numpy as np
from random import shuffle
import pandas as pd
from PIL import Image as im
from matplotlib import pyplot as plt
import torch.nn as nn
import torchvision.transforms as T
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
import cv2 as cv
from torch.utils.data.sampler import SubsetRandomSampler
import gc
import random

In [217]:
train_dir = '/kaggle/input/nn23-sports-image-classification/Train'
test_dir = '/kaggle/input/nn23-sports-image-classification/Test'
L = 0.0001 # perhaps change
ImgSize = 224
num_classes = 6
epochs = 500
batchSize = 64
G_Cdevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(G_Cdevice)

cuda


In [218]:

def create_label(image_name): # create label from image name
    word_label = image_name.split('_')[0]
    sports = ['Basketball', 'Football' , 'Rowing', 'Swimming', 'Tennis', 'Yoga']
    for s in range (len(sports)):
        if word_label == sports[s]:
            return s

        

In [219]:
def changeExtention():
    
    for img in tqdm(os.listdir(train_dir)):
        path = os.path.join(train_dir, img)
        ext = img.split('.')
        if ext[-1]=='png':
            img_data = im.open(path)
            img_data.save(os.path.join(new_path, ext[0]+'.jpg'))
        else:
            img_data = im.open(path)
            img_data.save(os.path.join(new_path, img))
# this function: change all png images to jpg images

In [220]:
new_dir = "new_train"
new_path = os.path.join("/kaggle/working", new_dir)
if not os.path.exists(new_path):
    os.mkdir(new_path, mode = 0o666)
    changeExtention()
    print("done converting")
print(new_path)



/kaggle/working/new_train


In [221]:
#global trainingData
#train_dir = new_path
def preproces():
    preprocess = T.Compose([
           T.Resize((ImgSize,ImgSize)),
#            T.CenterCrop(ImgSize),
           T.ToTensor(),
           T.Normalize(
               mean=[0.4914, 0.4822, 0.4465],
               std=[0.2023, 0.1994, 0.2010]
           )
    ])
    return preprocess


def LoadTrainingData():
    global training_data
    training_data = list()
    #trainingData = list()
    for img in tqdm(os.listdir(train_dir)):
        path = os.path.join(train_dir, img)
        img_data = im.open(path)       
        preproceses = preproces()
        img_data = preproceses(img_data)
        training_data.append([img_data, create_label(img)])
        #trainingData.append([img_data, create_label(img)])
    #trainingData = training_data
    #print(len(trainingData))
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data


def LoadTestingData():
    testing_data=[]
    imgName=[]
    for img in tqdm(os.listdir(test_dir)): 
        path = os.path.join(test_dir, img)
        imgName.append(img)
        img_data = im.open(path)
        preproceses = preproces()
        img_data = preproceses(img_data)
        testing_data.append(img_data)
#         shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    
    return testing_data, imgName


In [222]:

if (os.path.exists('train_data.npy')): # If we have already created the dataset:
    training_data =np.load('train_data.npy',allow_pickle=True)
#    train_dir = new_path
else: # If dataset is not created:

    training_data = LoadTrainingData()
    print(len(training_data))


In [223]:
#training_data = pd.DataFrame(training_data)
training = training_data



In [224]:

X_train = ([i[0] for i in training])
y_train = [i[1] for i in training]
split =1300
valSet=381
Xvalid = X_train[split:]
yvalid = y_train[split:]

yvalid =torch.tensor(yvalid,device='cuda')
yvalid =yvalid.reshape([valSet,1])


In [225]:
# split =1300
# valSet=381
# training = training_data
# testing = testing_data
# type(testing[0])
# Xtrain = ([i[0] for i in training])
# ytrain = [i[1] for i in training]

# Xvalid = Xtrain[split:]
# yvalid = ytrain[split:]

# yvalid =torch.tensor(yvalid,device='cuda')
# yvalid =yvalid.reshape([valSet,1])

# training = training_data[:split]#.tolist()


In [226]:
Horizontal_Flipping_Transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=1) 
])

Color_Transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ColorJitter(brightness=(0.1,0.6), contrast=1,saturation=0, hue=0.4)
    #transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5)
])

Vertical_Flipping_Transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomVerticalFlip(p=1) 
])

Rotate_Transformation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(degrees=80)
])
grayscale = transforms.Compose([  
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=1)
    ])

In [227]:




def make_augmentation(training_data):
    a = ((len(training_data)-1)/4)
    augment1=random.randint(0,a)
    print(augment1)
    augment2=random.randint(0,a)
    print(augment2)
    augment3=random.randint(0,a)
    print(augment3)
    augment4=random.randint(0,a)
    print(augment4)
    augment5=random.randint(0,a)
    print(augment5)

    print(len(training_data))
    #training_data_ = list()
    #training_data = training_data.tolist()
    i=0
    for img in tqdm(os.listdir(train_dir)):
        if i< augment1:
            img_data = cv.imread(os.path.join(train_dir, img))[:,:,::-1]
            Flipping_Img1 = Horizontal_Flipping_Transformation(img_data)
            preproceses_ = preproces()
            Flipping_Img1 = preproceses_(Flipping_Img1)
            training_data.append([Flipping_Img1, create_label(img)])
        #plt.imshow(Flipping_Img1)
        #plt.show()
#         elif i> augment1 & i < augment1+ augment2:
#             img_data = cv.imread(os.path.join(train_dir, img))[:,:,::-1]
#             Flipping_Img2 = Vertical_Flipping_Transformation(img_data)
#             preproceses_ = preproces()
#             Flipping_Img2 = preproceses_(Flipping_Img2)
#             training_data.append([Flipping_Img2, create_label(img)])
        #plt.imshow(Flipping_Img2)
        #plt.show()
        elif i > augment1 & i < augment1+ augment2:
            img_data = cv.imread(os.path.join(train_dir, img))[:,:,::-1]
            Transformed_Img = Color_Transformation(img_data)
            preproceses_ = preproces()
            Transformed_Img = preproceses_(Transformed_Img)
            training_data.append([Transformed_Img, create_label(img)])
        #plt.imshow(Transformed_Img)
        #plt.show()
        elif i> augment1+ augment2 & i < augment1+ augment2+ augment3:
            img_data = cv.imread(os.path.join(train_dir, img))[:,:,::-1]        
            Rotated_Img = Rotate_Transformation(img_data)
            preproceses_ = preproces()
            Rotated_Img = preproceses_(Rotated_Img)
            training_data.append([Rotated_Img, create_label(img)])
        elif i> augment1+ augment2 + augment3 & i < augment1+ augment2+ augment3+ augment4:
            img_data = cv.imread(os.path.join(train_dir, img))[:,:,::-1]        
            grayIMG = grayscale(img_data)
            preproceses_ = preproces()
            grayIMG = preproceses_(grayIMG)
            training_data.append([grayIMG, create_label(img)])
        #plt.imshow(Rotated_Img)
        #plt.show()    
        if i == augment1+ augment2+ augment3+ augment4:
            break
        i = i +1
        if i==200:
            break
    return training_data

if (os.path.exists('train_data12.npy')): # If we have already created the dataset:
    training_data =np.load('train_data12.npy',allow_pickle=True)
else:
    training_data = make_augmentation(training_data)
    shuffle(training_data)
    np.save('train_data12.npy', training_data)
    

print(len(training_data))
training = training_data


1881


In [228]:
X_train = ([i[0] for i in training])
y_train = [i[1] for i in training]
y_train =torch.tensor(y_train,device='cuda')
y_train =y_train.reshape([len(X_train),1])

In [229]:
class block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(block, self).__init__()
        self.expansion = 1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
     
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


In [230]:
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels=3, num_classes=6):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(block, layers[0],64, out_channels=64, stride=1)
        self.layer2 = self._make_layer(block, layers[1],64,out_channels=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2],128,out_channels=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3],256 ,out_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 1, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        #x = x.reshape(x.shape[0], -1)
        #x = x.view(2048)
        x = self.fc(x)

        return x
    def _make_layer(self, block, num_residual_blocks,in_channels, out_channels, stride):
        
        identity_downsample = None
        if stride != 1:
            identity_downsample = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1,bias=False), 
            nn.BatchNorm2d(out_channels)
        )
        layers=[]
        layers.append(block(in_channels, out_channels, identity_downsample, stride))
        layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    
#     def _make_layer(self, block, num_residual_blocks, out_channels, stride):
#         identity_downsample = None
        

#         # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
#         # we need to adapt the Identity (skip connection) so it will be able to be added
#         # to the layer that's ahead
        
#         #make sure of for loop (intermediate_channels * 4)
#         #use of downsample
#         if stride != 1 or self.in_channels != out_channels*1:
#             identity_downsample = nn.Sequential(
#                 nn.Conv2d(self.in_channels,out_channels * 1,kernel_size=1,stride = stride,bias=False),
#                 nn.BatchNorm2d(out_channels * 1),
#             )
#         layers = []
#         self.in_channels= out_channels
#         layers.append(block(self.in_channels, out_channels, identity_downsample, stride))

#         # The expansion size is always 4 for ResNet 50,101,152
#         #self.in_channels = out_channels * 1

#         # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
#         # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
#         # and also same amount of channels.
#         #self.in_channels = out_channels
#         for i in range(num_residual_blocks -1):
#             layers.append(block(self.in_channels,out_channels))

#         return nn.Sequential(*layers)

In [231]:
# img=Xtrain[4]
# img = img.permute(1, 2, 0).numpy()
# plt.imshow(img)
# print(ytrain[4])

In [232]:
# batch=random.randint(0,len(X_train_))
# batch

In [233]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()

# Train the model

def start_trainRes(ytrain_,Xtrain_):
    model = ResNet(block, [2, 2, 2, 2],3,6).to(G_Cdevice)
    #loss = nn.MSELoss()
    loss = nn.CrossEntropyLoss()
    #optimizer = torch.optim.Adam(model.parameters(), lr=L,  weight_decay = 0.000000001)
    optimizer = torch.optim.SGD(model.parameters(), lr=L, weight_decay = 0.001, momentum = 0.9)  
    global output
    print(len(Xtrain_))
    total=0
    correct=0
    for each_epoch in range(epochs):
        batch=random.randint(0,len(Xtrain_))
        b=0
        global error
        for ind in range(batch,len(Xtrain_)):
            if b == batchSize:
                break
            b+=1 
            Xtrain_[ind] = Xtrain_[ind].to(G_Cdevice)
            ytrain_[ind] = ytrain_[ind].to(G_Cdevice)
            output = model.forward(Xtrain_[ind].unsqueeze(0))
            error = loss(output,ytrain_[ind])
            optimizer.zero_grad()
            error.backward()
            optimizer.step()
#             _, predicted = torch.max(output.data, 1)
#             total += ytrain_[i].size(0)
#             correct += (predicted == ytrain_[i].to(G_Cdevice)).sum().item()
#             del X_train[ind], y_train_[ind], output
#             torch.cuda.empty_cache()
#             gc.collect()
        print("Epoch: ",each_epoch)
        print("Error: ",error.item())
    return model    

modelR = start_trainRes(y_train,X_train)


1881
Epoch:  0
Error:  1.2446298599243164
Epoch:  1
Error:  2.17830753326416
Epoch:  2
Error:  1.7874705791473389
Epoch:  3
Error:  2.0429420471191406
Epoch:  4
Error:  1.4447808265686035
Epoch:  5
Error:  1.7806296348571777
Epoch:  6
Error:  1.3185456991195679
Epoch:  7
Error:  1.6900146007537842
Epoch:  8
Error:  0.7237643003463745
Epoch:  9
Error:  1.639716386795044
Epoch:  10
Error:  1.219748616218567
Epoch:  11
Error:  1.8804198503494263
Epoch:  12
Error:  2.057985305786133
Epoch:  13
Error:  2.1625375747680664
Epoch:  14
Error:  1.8924392461776733
Epoch:  15
Error:  0.9017083644866943
Epoch:  16
Error:  0.49274832010269165
Epoch:  17
Error:  1.3548816442489624
Epoch:  18
Error:  0.4167132079601288
Epoch:  19
Error:  0.8575946092605591
Epoch:  20
Error:  0.9448845386505127
Epoch:  21
Error:  0.24890705943107605
Epoch:  22
Error:  0.25270015001296997
Epoch:  23
Error:  0.39513739943504333
Epoch:  24
Error:  1.5763561725616455
Epoch:  25
Error:  0.7665671110153198
Epoch:  26
Error: 

In [234]:

correct = 0
total = 0
for i in range(len(Xvalid)):
    Xvalid[i] = Xvalid[i].to(G_Cdevice)
    yvalid[i] = yvalid[i].to(G_Cdevice)
    outputs = modelR.forward(Xvalid[i].unsqueeze(0))
    _, predicted = torch.max(outputs.data, 1)
    total += yvalid[i].size(0)
    correct += (predicted == yvalid[i].to(G_Cdevice)).sum().item()
   # del images, labels, outputs

print('Accuracy on validation images: {} %'
      .format(100 * correct / total)) 

Accuracy on validation images: 100.0 %


In [235]:
# Xtest = ([i[0] for i in testing])
# img=Xtest[4]
# img = img.permute(-2,1).numpy()
# plt.imshow(img)
# print(ytrain[4])

In [236]:
# global output 

# for i in range(len(Xtest)):
#     Xtest[i] = Xtest[i].to(G_Cdevice)
#     outputs = modelR.forward(Xtest[i].unsqueeze(0))
#     _, predicted = torch.max(outputs.data, 1)
outputPred = []
imageName = []

for img in tqdm(os.listdir(test_dir)): 
    path = os.path.join(test_dir, img)
    imageName.append(img)
    img_data = im.open(path)
    preproceses = preproces()
    img_data = preproceses(img_data)
    img_data = img_data.to(G_Cdevice)
    outputs = modelR.forward(img_data.unsqueeze(0))
    _, predicted = torch.max(outputs.data, 1)
    outputPred.append(predicted.item())

# for i in range(len(testing)):
#     testing[i] = testing[i].to(G_Cdevice)
#     outputs = modelR.forward(testing[i].unsqueeze(0))
#     _, predicted = torch.max(outputs.data, 1)
# #     print(predicted.item())
#     outputPred.append(predicted.item())
#     imageName.append(str(i)+'.jpg')

100%|██████████| 688/688 [00:14<00:00, 46.88it/s]


In [237]:
data = pd.DataFrame()
data['image_name'] = imageName
data['label'] = outputPred
#data.to_csv("team11_CS.csv")
data.to_csv('team11_CS.csv', index=False)

data

,image_name,label
0,623.jpg,4
1,208.jpg,5
2,473.jpg,0
3,333.jpg,1
4,537.jpg,1
...,...,...
683,364.jpg,3
684,90.jpg,3
685,599.jpg,4
686,25.jpg,1
